# 常用损失函数小案例

In [1]:
import tensorflow as tf


In [3]:
cce = tf.keras.losses.CategoricalCrossentropy()
loss = cce(
  [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]],
  [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
print('Loss: ', loss.numpy())  # Loss: 0.0945

Loss:  0.09458993


我们手动算下loss是否正确

In [10]:
a =np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]],)
b =np.array([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
np.average(-np.sum(a*np.log(b),axis=1))


0.09458991187728844

# 自定义损失函数小案例


我们来实现实现mse这个损失函数，看看是怎么自定义损失函数的

In [27]:
#类
class MeanSquaredError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_pred - y_true))

In [18]:
a = tf.constant([0., 0., 1., 1.])
b = tf.constant([1., 1., 1., 0.])

In [19]:
mse = MeanSquaredError()

In [20]:
mse(a,b)

<tf.Tensor: id=80, shape=(), dtype=float32, numpy=0.75>

In [26]:
# 函数式

In [22]:
def MeanSquaredError( y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

In [24]:
mse1 = MeanSquaredError(a,b)

In [25]:
mse1

<tf.Tensor: id=84, shape=(), dtype=float32, numpy=0.75>

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
loss = mse([0., 0., 1., 1.], [1., 1., 1., 0.])
print('Loss: ', loss.numpy())  # Loss: 0.75

# Focal loss

In [19]:
import numpy as np
a =np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
b =np.array([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
-np.sum(a*np.log(b),axis=1)


array([0.10536052, 0.11653382, 0.0618754 ])

In [20]:
np.average(np.sum(-np.power(1-b,2)*a*np.log(b),axis=1))

0.000895471928886797

In [32]:
#多分类的focal loss 损失函数
class SparseFocalLoss(tf.keras.losses.Loss):

    def __init__(self,gamma=2.0,alpha=0.25,class_num=10):
        self.gamma = gamma
        self.alpha = alpha
        self.class_num = class_num 
        super(SparseFocalLoss, self).__init__()

    def call(self,y_true,y_pred):
#         y_pred = tf.nn.softmax(y_pred,axis=-1)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0)
        
        y_true = tf.one_hot(y_true,depth=self.class_num)
        y_true = tf.cast(y_true,tf.float32)
        
        loss = -  y_true * tf.math.pow(1 - y_pred, self.gamma) * tf.math.log(y_pred)
        
        loss = tf.math.reduce_sum(loss,axis=1)
        return loss
    

In [33]:
focalloss = SparseFocalLoss(gamma=2.0,alpha=0.25,class_num=3)

In [34]:
focalloss(tf.constant([0,1,2]),
          tf.constant([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]))

<tf.Tensor: id=283, shape=(), dtype=float32, numpy=0.0008954724>